<a href="https://colab.research.google.com/github/anna-marshalova/model-compression-course/blob/main/mbert_compression_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer

MODEL_NAME = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
from torch.nn import Module, Linear
from transformers import AutoModel


class Model(Module):
    def __init__(self, pretrained_model_name, num_labels, freeze = True, **kwargs):
        super().__init__()
        self.emb = AutoModel.from_pretrained(pretrained_model_name, output_attentions=False, output_hidden_states=False)
        if freeze:
            for param in self.emb.parameters():
                param.requires_grad = False
        hid_size = list(self.emb.parameters())[-1].shape[0]
        self.out = Linear(hid_size, num_labels)

    def forward(self, input_ids, attention_mask, **kwargs):
        x = self.emb(input_ids = input_ids, attention_mask = attention_mask).pooler_output
        return self.out(x)

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
CHECKPOINTS_PATH = '/content/drive/MyDrive/model_compression/model.pt'
model = torch.load(CHECKPOINTS_PATH, map_location = device)

In [ ]:
import inference
from importlib import reload
reload(inference)

<module 'inference' from '/content/inference.py'>

In [ ]:
from inference import get_test_data

In [ ]:
test_dataset, test_loader = get_test_data()

Generating train split:   0%|          | 0/270399 [00:00<?, ? examples/s]

----------------------------------------------------------------------------------------------------
                                                text     label  source  \
0    yang memerlukan pemerhatian dan tindakan serius  positive  malaya   
1  sentiasa memikirkan dan merancang inisiatif ba...  positive  malaya   
2  Kita akan tengok daripada pelbagai aspek supay...  positive  malaya   
3  justeru asean perlu mengambil tindakan sebagai...  positive  malaya   
4  @_Niiar_ Jangan punah dulu, aku belum ke labua...   neutral  malaya   

  language  
0    malay  
1    malay  
2    malay  
3    malay  
4    malay  
----------------------------------------------------------------------------------------------------


Generating validation split:   0%|          | 0/10857 [00:00<?, ? examples/s]

----------------------------------------------------------------------------------------------------
                                                text     label  source  \
0  kerja sama apec perlu wujudkan pertumbuhan eko...  positive  malaya   
1    Ada yang disembunyikan, dan itu yang dipercayai  negative  malaya   
2  Apatah lagi penduduk negara ini terdiri daripa...  positive  malaya   
3  tetapi sejak krisis kewangan asia hingga sekar...  negative  malaya   
4  KOTA KINABALU: Warga emas tertua di Sabah mene...  positive  malaya   

  language  
0    malay  
1    malay  
2    malay  
3    malay  
4    malay  
----------------------------------------------------------------------------------------------------


Generating test split:   0%|          | 0/14465 [00:00<?, ? examples/s]

----------------------------------------------------------------------------------------------------
                                                text     label  source  \
0  Sepatutnya berbuat begitu  demi untuk menawark...  positive  malaya   
1  Alhamdulillah, sama2 bantu kerajaan memerangi ...  positive  malaya   
2  Biasanya bantuan disalurkan kepada sekolah ber...  positive  malaya   
3  Kerajaan wajar mengkaji semula had tunggakan c...  positive  malaya   
4  me; those everytime nak beli baju or seluar ta...  positive  malaya   

  language  
0    malay  
1    malay  
2    malay  
3    malay  
4    malay  
----------------------------------------------------------------------------------------------------


In [ ]:
from inference import eval_all_langs, scores_to_df, measure_size_mb, measure_inference_time

In [ ]:
test_langs = set(dataset['test']['language'])
scores = eval_all_langs(test_dataset, model, test_langs)

In [ ]:
scores_to_df(scores)

,language,f1
0,chinese,0.556626
1,arabic,0.344552
2,portuguese,0.369250
3,japanese,0.499123
4,malay,0.565781
5,english,0.343242
6,hindi,0.359026
7,german,0.316250
8,indonesian,0.327033
9,spanish,0.272859


In [ ]:
measure_size_mb(model)

Model size: 678.474MB


In [ ]:
measure_inference_time(model, test_loader)

  0%|          | 0/46 [00:00<?, ?it/s]

Avg inference time: 18.049402236938477
